In [21]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

In [22]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [23]:
mlp_baseline_results = pd.read_csv('mlp_baseline_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)

mlp_pearson_results = pd.read_csv('mlp_pearson_results.csv')
mlp_pearson_results.replace({'mlp_preg_initializer': 'Pearson regularized MLP'}, inplace=True)

mlp_spearman_results = pd.read_csv('mlp_spearman_results.csv')
mlp_spearman_results.replace({'mlp_sreg_initializer': 'Spearman regularized MLP'}, inplace=True)

mlp_results = pd.concat([mlp_baseline_results, mlp_pearson_results, mlp_spearman_results])

ftl_baseline_results = pd.read_csv('ftl_baseline_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': 'Standard FTL (baseline)'}, inplace=True)

ftl_pearson_results = pd.read_csv('ftl_pearson_results.csv')
ftl_pearson_results.replace({'ftl_mlp_initializer': 'Pearson regularized FTL'}, inplace=True)

ftl_spearman_results = pd.read_csv('ftl_spearman_results.csv')
ftl_spearman_results.replace({'ftl_mlp_initializer': 'Spearman regularized FTL'}, inplace=True)

ftl_results = pd.concat([ftl_baseline_results, ftl_pearson_results, ftl_spearman_results])

In [24]:
for results in [mlp_results,ftl_results]:
    results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
    results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [25]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

for results in [mlp_results,ftl_results]:
    results['Performance'] = 0
    results['Fairness'] = 0
    results['Fitness Rule'] = ''
    for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
        results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_1373795/3607436483.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.56841327
 0.56481476 0.58141906 0.57136131 0.58334192 0.55288815 0.56058845
 0.5737951  0.57000708 0.57640052 0.58261133 0.57454722 0.57134873
 0.59039745 0.58475101 0.58230021 0.57479453 0.5876658  0.59199945
 0.59145802 0.5797924  0.59391819]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
/tmp/ipykernel_1373795/3607436483.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.19263281 0.18664775 0.20096652 0.19289642 0.19661682 0.17761564
 0.19442911 0.1975522

In [27]:
display(mlp_results)

,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,solution,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,Adult Income,Standard MLP (baseline),mcc_parity,0.392537,0.851299,0.585170,0.113127,0.192633,0.142532,{'dropout': 0.16228954240968418},0.585170,0.192633,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,Adult Income,Standard MLP (baseline),mcc_odds,0.473935,0.851520,0.585500,0.111565,0.198915,0.132052,{'dropout': 0.1482543145261793},0.585500,0.111565,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,Adult Income,Standard MLP (baseline),mcc_opportunity,0.460175,0.851410,0.585064,0.103235,0.191606,0.124889,{'dropout': 0.17861515209545933},0.585064,0.124889,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,Adult Income,Standard MLP (baseline),acc_parity,0.661796,0.844887,0.565373,0.082386,0.183090,0.084174,{'dropout': 0.15655371267306142},0.844887,0.183090,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,Adult Income,Standard MLP (baseline),acc_odds,0.766170,0.850857,0.577932,0.084687,0.180366,0.093891,{'dropout': 0.1698638434734938},0.850857,0.084687,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Adult Income,Spearman regularized MLP,mcc_odds,0.474808,0.849862,0.584044,0.109236,0.197334,0.135481,"{'l2': 0.001, 'dropout': 0.1358582308261664}",0.584044,0.109236,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
62,Adult Income,Spearman regularized MLP,mcc_opportunity,0.529244,0.852847,0.584568,0.068332,0.181148,0.055323,"{'l2': 0.001, 'dropout': 0.046815908519132345}",0.584568,0.055323,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
63,Adult Income,Spearman regularized MLP,acc_parity,0.659911,0.846877,0.579077,0.088519,0.186966,0.094479,"{'l2': 0.001, 'dropout': 0.02420010661774878}",0.846877,0.186966,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
64,Adult Income,Spearman regularized MLP,acc_odds,0.806630,0.849530,0.575692,0.042900,0.164714,0.014491,"{'l2': 0.001, 'dropout': 0.07752729427158084}",0.849530,0.042900,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)


In [28]:
datasets = ['Adult Income']#, 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income']

In [29]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [33]:
ftl_methods = ['Standard FTL (baseline)', 'Pearson regularized FTL', 'Spearman regularized FTL']
mlp_methods = ['Standard MLP (baseline)', 'Pearson regularized MLP', 'Spearman regularized MLP']

In [34]:
for path, methods, results in zip(['mlp_multi_aso_data_list.json', 'ftl_multi_aso_data_list.json'],
                                  [mlp_methods, ftl_methods],
                                  (mlp_results,ftl_results)):
    if os.path.exists(path):
        with open(path) as file:
            multi_aso_data_list = json.load(file)
    else:    
        multi_aso_data_list = []
        for d in datasets:
            multi_aso_data = []
            for f in fitness_rules:
                methods_results = []
                for m in methods:
                    r = results.loc[ (results['dataset'] == d) &
                                         (results['fitness_rule'] == f) &
                                         (results['method'] == m) ]\
                                .fitness.tolist()
                    if len(r) == 0:
                        r = [-1]
                    methods_results.append(r)
                min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
                multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
        with open(path, 'w') as file:
            json.dump(multi_aso_data_list, file)
    
    aso_df_resume = []
    for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
        fitness_rule = aso_result['fitness_rule']
        dataset = aso_result['dataset']
    
        aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
        aso_df['method'] = methods
        aso_df['dataset'] = dataset
        aso_df['fitness_rule'] = fitness_rule
        aso_df_resume.append(aso_df)
    
    print('Significance Testing')
    significance = pd.concat(aso_df_resume)
    significance.replace(fitness_rules_abvr, inplace=True)
    significance = significance.set_index(['fitness_rule', 'dataset'])
    significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
    display(significance)
    
    grouped_results = results\
        .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
        .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
        .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
    grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
    grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
    display(grouped_results)

Model comparisons: 100%|█████████▉| 2997/3000 [00:07<00:00, 398.40it/s]

Significance Testing


Standard MLP (baseline)  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 0.709022   
                        Adult Income                 0.039775   
Max(MCC - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 0.168983   
                        Adult Income                 0.548240   
Max(MCC - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.589897   
Max(Acc - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(Acc - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.707744   
Max(Acc - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   

                                      Pearson regularized MLP  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.246732   
Max(MCC - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(MCC - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.583747   
Max(Acc - Stat. Parity) Adult Income                 0.213667   
                        Adult Income                 1.000000   
                        Adult Income                 0.411649   
Max(Acc - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.712637   
Max(Acc - Eq. Odds)     Adult Income                 0.748260   
                        Adult Income                 1.000000   
                        Adult Income                 0.775861   

                                      Spearman regularized MLP  \
fitness_rule            dataset                                  
Max(MCC - Stat. Parity) Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(MCC - Eq. Opp.)     Adult Income                  1.000000   
                        Adult Income                  0.512791   
                        Adult Income                  1.000000   
Max(MCC - Eq. Odds)     Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(Acc - Stat. Parity) Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(Acc - Eq. Opp.)     Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(Acc - Eq. Odds)     Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   

                                                        method  
fitness_rule            dataset                   

fitness  \
                                                                   mean   
Fitness Rule Abvr       dataset      method                               
Max(Acc - Eq. Odds)     Adult Income Standard MLP (baseline)   0.761670   
                                     Spearman regularized MLP  0.760737   
                                     Pearson regularized MLP   0.749159   
Max(Acc - Eq. Opp.)     Adult Income Spearman regularized MLP  0.764593   
                                     Pearson regularized MLP   0.759218   
                                     Standard MLP (baseline)   0.759144   
Max(Acc - Stat. Parity) Adult Income Standard MLP (baseline)   0.663749   
                                     Spearman regularized MLP  0.659905   
                                     Pearson regularized MLP   0.652003   
Max(MCC - Eq. Odds)     Adult Income Spearman regularized MLP  0.478951   
                                     Standard MLP (baseline)   0.466962   
                                     Pearson regularized MLP   0.465810   
Max(MCC - Eq. Opp.)     Adult Income Pearson regularized MLP   0.497117   
                                     Spearman regularized MLP  0.486588   
                                     Standard MLP (baseline)   0.479067   
Max(MCC - Stat. Parity) Adult Income Spearman regularized MLP  0.397889   
                                     Pearson regularized MLP   0.388343   
                                     Standard MLP (baseline)   0.384943   

                                                                               \
                                                                    std count   
Fitness Rule Abvr       dataset      method                                     
Max(Acc - Eq. Odds)     Adult Income Standard MLP (baseline)   0.036923    11   
                                     Spearman regularized MLP  0.044149    11   
                                     Pearson regularized MLP   0.015001    11   
Max(Acc - Eq. Opp.)     Adult Income Spearman regularized MLP  0.023150    11   
                                     Pearson regularized MLP   0.021135    11   
                                     Standard MLP (baseline)   0.020645    11   
Max(Acc - Stat. Parity) Adult Income Standard MLP (baseline)   0.010845    11   
                                     Spearman regularized MLP  0.015297    11   
                                     Pearson regularized MLP   0.015825    11   
Max(MCC - Eq. Odds)     Adult Income Spearman regularized MLP  0.046799    11   
                                     Standard MLP (baseline)   0.028307    11   
                                     Pearson regularized MLP   0.037051    11   
Max(MCC - Eq. Opp.)     Adult Income Pearson regularized MLP   0.019964    11   
                                     Spearman regularized MLP  0.022377    11   
                                     Standard MLP (baseline)   0.018527    11   
Max(MCC - Stat. Parity) Adult Income Spearman regularized MLP  0.010582    11   
                                     Pearson regularized MLP   0.014920    11   
                                     Standard MLP (baseline)   0.009809    11   

                                                              Performance  \
                                                                     mean   
Fitness Rule Abvr       dataset      method                                 
Max(Acc - Eq. Odds)     Adult Income Standard MLP (baseline)     0.846495   
                                     Spearman regularized MLP    0.847098   
                                     Pearson regularized MLP     0.849319   
Max(Acc - Eq. Opp.)     Adult Income Spearman regularized MLP    0.847359   
                                     Pearson regularized MLP     0.846746   
                                     Standard MLP (baseline)     0.849379   
Max(Acc - Stat. Parity) Adult Income Standard MLP (baseline)     0.849912   
                

Model comparisons: 100%|█████████▉| 2997/3000 [00:07<00:00, 394.02it/s]

Significance Testing


Standard FTL (baseline)  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 0.579539   
                        Adult Income                 1.000000   
Max(MCC - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 0.358964   
                        Adult Income                 0.453264   
Max(MCC - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(Acc - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 0.187643   
                        Adult Income                 0.422696   
Max(Acc - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 0.614676   
                        Adult Income                 0.569497   
Max(Acc - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 0.902173   
                        Adult Income                 1.000000   

                                      Pearson regularized FTL  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(MCC - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(MCC - Eq. Odds)     Adult Income                 0.177844   
                        Adult Income                 1.000000   
                        Adult Income                 0.472770   
Max(Acc - Stat. Parity) Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   
Max(Acc - Eq. Opp.)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 0.983148   
Max(Acc - Eq. Odds)     Adult Income                 1.000000   
                        Adult Income                 1.000000   
                        Adult Income                 1.000000   

                                      Spearman regularized FTL  \
fitness_rule            dataset                                  
Max(MCC - Stat. Parity) Adult Income                  0.371769   
                        Adult Income                  0.246382   
                        Adult Income                  1.000000   
Max(MCC - Eq. Opp.)     Adult Income                  1.000000   
                        Adult Income                  0.665979   
                        Adult Income                  1.000000   
Max(MCC - Eq. Odds)     Adult Income                  0.602087   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(Acc - Stat. Parity) Adult Income                  1.000000   
                        Adult Income                  0.504629   
                        Adult Income                  1.000000   
Max(Acc - Eq. Opp.)     Adult Income                  1.000000   
                        Adult Income                  1.000000   
                        Adult Income                  1.000000   
Max(Acc - Eq. Odds)     Adult Income                  0.270573   
                        Adult Income                  0.198452   
                        Adult Income                  1.000000   

                                                        method  
fitness_rule            dataset                   

fitness  \
                                                                   mean   
Fitness Rule Abvr       dataset      method                               
Max(Acc - Eq. Odds)     Adult Income Pearson regularized FTL   0.816440   
                                     Standard FTL (baseline)   0.813400   
                                     Spearman regularized FTL  0.793284   
Max(Acc - Eq. Opp.)     Adult Income Pearson regularized FTL   0.806716   
                                     Spearman regularized FTL  0.805827   
                                     Standard FTL (baseline)   0.801817   
Max(Acc - Stat. Parity) Adult Income Pearson regularized FTL   0.813380   
                                     Spearman regularized FTL  0.809098   
                                     Standard FTL (baseline)   0.799946   
Max(MCC - Eq. Odds)     Adult Income Standard FTL (baseline)   0.552515   
                                     Spearman regularized FTL  0.542470   
                                     Pearson regularized FTL   0.527102   
Max(MCC - Eq. Opp.)     Adult Income Pearson regularized FTL   0.530239   
                                     Spearman regularized FTL  0.526425   
                                     Standard FTL (baseline)   0.516256   
Max(MCC - Stat. Parity) Adult Income Pearson regularized FTL   0.493097   
                                     Standard FTL (baseline)   0.486983   
                                     Spearman regularized FTL  0.475760   

                                                                               \
                                                                    std count   
Fitness Rule Abvr       dataset      method                                     
Max(Acc - Eq. Odds)     Adult Income Pearson regularized FTL   0.024623    16   
                                     Standard FTL (baseline)   0.021986    13   
                                     Spearman regularized FTL  0.036451    12   
Max(Acc - Eq. Opp.)     Adult Income Pearson regularized FTL   0.015056    16   
                                     Spearman regularized FTL  0.011884    12   
                                     Standard FTL (baseline)   0.015775    14   
Max(Acc - Stat. Parity) Adult Income Pearson regularized FTL   0.011659    16   
                                     Spearman regularized FTL  0.012363    12   
                                     Standard FTL (baseline)   0.023751    14   
Max(MCC - Eq. Odds)     Adult Income Standard FTL (baseline)   0.029883    14   
                                     Spearman regularized FTL  0.035399    12   
                                     Pearson regularized FTL   0.033794    16   
Max(MCC - Eq. Opp.)     Adult Income Pearson regularized FTL   0.015378    16   
                                     Spearman regularized FTL  0.014108    12   
                                     Standard FTL (baseline)   0.030559    14   
Max(MCC - Stat. Parity) Adult Income Pearson regularized FTL   0.021600    16   
                                     Standard FTL (baseline)   0.018719    14   
                                     Spearman regularized FTL  0.022828    12   

                                                              Performance  \
                                                                     mean   
Fitness Rule Abvr       dataset      method                                 
Max(Acc - Eq. Odds)     Adult Income Pearson regularized FTL     0.844002   
                                     Standard FTL (baseline)     0.847098   
                                     Spearman regularized FTL    0.843265   
Max(Acc - Eq. Opp.)     Adult Income Pearson regularized FTL     0.843954   
                                     Spearman regularized FTL    0.842500   
                                     Standard FTL (baseline)     0.839035   
Max(Acc - Stat. Parity) Adult Income Pearson regularized FTL     0.828372   
                